---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the Jupyter Notebook FAQ course resource._

---

# Assignment 4 - Predicting and understanding viewer engagement with educational videos 

With the accelerating popularity of online educational experiences, the role of online lectures and other educational video continues to increase in scope and importance. Open access educational repositories such as <a href="http://videolectures.net/">videolectures.net</a>, as well as Massive Open Online Courses (MOOCs) on platforms like Coursera, have made access to many thousands of lectures and tutorials an accessible option for millions of people around the world. Yet this impressive volume of content has also led to a challenge in how to find, filter, and match these videos with learners. This assignment gives you an example of how machine learning can be used to address part of that challenge.

## About the prediction problem

One critical property of a video is engagement: how interesting or "engaging" it is for viewers, so that they decide to keep watching. Engagement is critical for learning, whether the instruction is coming from a video or any other source. There are many ways to define engagement with video, but one common approach is to estimate it by measuring how much of the video a user watches. If the video is not interesting and does not engage a viewer, they will typically abandon it quickly, e.g. only watch 5 or 10% of the total. 

A first step towards providing the best-matching educational content is to understand which features of educational material make it engaging for learners in general. This is where predictive modeling can be applied, via supervised machine learning. For this assignment, your task is to predict how engaging an educational video is likely to be for viewers, based on a set of features extracted from the video's transcript, audio track, hosting site, and other sources.

We chose this prediction problem for several reasons:

* It combines a variety of features derived from a rich set of resources connected to the original data;
* The manageable dataset size means the dataset and supervised models for it can be easily explored on a wide variety of computing platforms;
* Predicting popularity or engagement for a media item, especially combined with understanding which features contribute to its success with viewers, is a fun problem but also a practical representative application of machine learning in a number of business and educational sectors.


## About the dataset

We extracted training and test datasets of educational video features from the VLE Dataset put together by researcher Sahan Bulathwela at University College London. 

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single educational video, and includes information about diverse properties of the video content as described further below. The target variable is `engagement` which was defined as True if the median percentage of the video watched across all viewers was at least 30%, and False otherwise.

Note: Any extra variables that may be included in the training set are simply for your interest if you want an additional source of data for visualization, or to enable unsupervised and semi-supervised approaches. However, they are not included in the test set and thus cannot be used for prediction. **Only the data already included in your Coursera directory can be used for training the model for this assignment.**

For this final assignment, you will bring together what you've learned across all four weeks of this course, by exploring different prediction models for this new dataset. In addition, we encourage you to apply what you've learned about model selection to do hyperparameter tuning using training/validation splits of the training data, to optimize the model and further increase its performance. In addition to a basic evaluation of model accuracy, we've also provided a utility function to visualize which features are most and least contributing to the overall model performance.

**File descriptions** 
    assets/train.csv - the training set (Use only this data for training your model!)
    assets/test.csv - the test set
<br>

**Data fields**

train.csv & test.csv:

    title_word_count - the number of words in the title of the video.
    
    document_entropy - a score indicating how varied the topics are covered in the video, based on the transcript. Videos with smaller entropy scores will tend to be more cohesive and more focused on a single topic.
    
    freshness - The number of days elapsed between 01/01/1970 and the lecture published date. Videos that are more recent will have higher freshness values.
    
    easiness - A text difficulty measure applied to the transcript. A lower score indicates more complex language used by the presenter.
    
    fraction_stopword_presence - A stopword is a very common word like 'the' or 'and'. This feature computes the fraction of all words that are stopwords in the video lecture transcript.
    
    speaker_speed - The average speaking rate in words per minute of the presenter in the video.
    
    silent_period_rate - The fraction of time in the lecture video that is silence (no speaking).
    
train.csv only:
    
    engagement - Target label for training. True if learners watched a substantial portion of the video (see description), or False otherwise.
    

## Evaluation

Your predictions will be given as the probability that the corresponding video will be engaging to learners.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model with an AUC (area under ROC curve) of at least 0.8 passes this assignment, and over 0.85 will receive full points.
___

For this assignment, create a function that trains a model to predict significant learner engagement with a video using `asset/train.csv`. Using this model, return a Pandas Series object of length 2309 with the data being the probability that each corresponding video from `readonly/test.csv` will be engaging (according to a model learned from the 'engagement' label in the training set), and the video index being in the `id` field.

Example:

    id
       9240    0.401958
       9241    0.105928
       9242    0.018572
                 ...
       9243    0.208567
       9244    0.818759
       9245    0.018528
             ...
       Name: engagement, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out and check your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. 

* Try to avoid global variables. If you have other functions besides engagement_model, you should move those functions inside the scope of engagement_model.

* Be sure to first check the pinned threads in Week 4's discussion forum if you run into a problem you can't figure out.

### Extensions

* If this prediction task motivates you to explore further, you can find more details here on the original VLE dataset and others related to video engagement: https://github.com/sahanbull/VLE-Dataset



In [1]:
import warnings
warnings.filterwarnings("ignore")

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(0)   # Do not change this value: required to be compatible with solutions generated by the autograder.

In [2]:
import pandas as pd

# Load the data
train_df = pd.read_csv('assets/train.csv')
test_df = pd.read_csv('assets/test.csv')

# Display the first few rows of the training data
print(train_df.head())

   id  title_word_count  document_entropy  freshness   easiness  \
0   1                 9          7.753995      16310  75.583936   
1   2                 6          8.305269      15410  86.870523   
2   3                 3          7.965583      15680  81.915968   
3   4                 9          8.142877      15610  80.148937   
4   5                 9          8.161250      14920  76.907549   

   fraction_stopword_presence  normalization_rate  speaker_speed  \
0                    0.553664            0.034049       2.997753   
1                    0.584498            0.018763       2.635789   
2                    0.605685            0.030720       2.538095   
3                    0.593664            0.016873       2.259055   
4                    0.581637            0.023412       2.420000   

   silent_period_rate  engagement  
0                 0.0        True  
1                 0.0       False  
2                 0.0       False  
3                 0.0       False  
4       

In [3]:
# Check for missing values
print(train_df.isnull().sum())

# Basic statistics of the training data
print(train_df.describe())

# Check the balance of the target variable
print(train_df['engagement'].value_counts(normalize=True))

id                            0
title_word_count              0
document_entropy              0
freshness                     0
easiness                      0
fraction_stopword_presence    0
normalization_rate            0
speaker_speed                 0
silent_period_rate            0
engagement                    0
dtype: int64
                id  title_word_count  document_entropy     freshness  \
count  9239.000000       9239.000000       9239.000000   9239.000000   
mean   4620.000000          7.701050          7.792685  14808.587509   
std    2667.213902          3.785066          0.697710   1208.953646   
min       1.000000          1.000000          0.000000  10830.000000   
25%    2310.500000          5.000000          7.594235  14070.000000   
50%    4620.000000          7.000000          7.875103  14750.000000   
75%    6929.500000         10.000000          8.164166  15600.000000   
max    9239.000000         33.000000          9.278573  17430.000000   

          easiness

In [4]:
from sklearn.model_selection import train_test_split

X = train_df.drop(columns=['engagement'])
y = train_df['engagement']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0)

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

# Train the model
model = LogisticRegression(random_state=0)
model.fit(X_train, y_train)

# Evaluate the model
y_val_pred = model.predict_proba(X_val)[:, 1]
print('Validation AUC:', roc_auc_score(y_val, y_val_pred))

Validation AUC: 0.8128588823378621


In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

# Train the model
model = RandomForestClassifier(random_state=0)
model.fit(X_train, y_train)

# Evaluate the model
y_val_pred = model.predict_proba(X_val)[:, 1]
print('Validation AUC:', roc_auc_score(y_val, y_val_pred))

Validation AUC: 0.8776836067674956


In [10]:
from sklearn.model_selection import GridSearchCV

param_grid = {'C': [0.1, 1, 10, 100]}
grid_search = GridSearchCV(LogisticRegression(random_state=0), param_grid, scoring='roc_auc', cv=5)
grid_search.fit(X_train, y_train)

# Best model
best_model = grid_search.best_estimator_
y_val_pred = best_model.predict_proba(X_val)[:, 1]
print('Best Validation AUC:', roc_auc_score(y_val, y_val_pred))

Best Validation AUC: 0.8130158933606767


In [16]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score

def engagement_model():
    # Load the data
    train_df = pd.read_csv('assets/train.csv')
    test_df = pd.read_csv('assets/test.csv')
    
    # Separate features and target
    X = train_df.drop(columns=['engagement'])
    y = train_df['engagement']
    
    # Preprocess the data: Scale the features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    test_scaled = scaler.transform(test_df)
    
    # Train the model
    model = LogisticRegression(random_state=0)
    model.fit(X_scaled, y)
    
    # Make predictions on the test set
    test_pred = model.predict_proba(test_scaled)[:, 1]
    
    # Create the result series with video IDs
    result = pd.Series(test_pred, index=test_df.index, name='engagement')
    
    return result

# Call the function to ensure it works correctly
stu_ans = engagement_model()
print(stu_ans.head())

# Assertions to check the output
assert isinstance(stu_ans, pd.Series), "Your function should return a pd.Series."
assert len(stu_ans) == 2309, "Your series is of incorrect length: expected 2309."
assert np.issubdtype(stu_ans.index.dtype, np.integer), "Your answer pd.Series should have an index of integer type representing video id."

0    0.011781
1    0.091383
2    0.100932
3    0.745624
4    0.018466
Name: engagement, dtype: float64


In [30]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score

def engagement_model():
    # Load the data
    train_df = pd.read_csv('assets/train.csv')
    test_df = pd.read_csv('assets/test.csv')

    # Separate the features and target variable
    X = train_df.drop(columns=['engagement', 'id'])  # Drop 'engagement' and 'id' columns
    y = train_df['engagement']

    # Split into training and validation sets
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0)

    # Preprocess the data: Scale the features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)
    test_scaled = scaler.transform(test_df.drop(columns=['id']))  # Ensure 'id' is dropped

    # Train a Random Forest model with GridSearchCV
    param_grid = {
        'n_estimators': [50, 100],
        'max_depth': [10, None],
        'min_samples_split': [2, 5],
        'min_samples_leaf': [1, 2],
        'bootstrap': [True]
    }
    grid_search = GridSearchCV(RandomForestClassifier(random_state=0), param_grid, scoring='roc_auc', cv=5)
    grid_search.fit(X_train_scaled, y_train)

    # Best model from GridSearchCV
    best_model = grid_search.best_estimator_

    # Predict on the test set
    test_pred = best_model.predict_proba(test_scaled)[:, 1]

    # Ensure the length of predictions matches the number of test samples
    len(test_pred) == len(test_df)

    # Create the result series with video IDs and set the data type to float32
    result = pd.Series(test_pred, index=test_df['id'], dtype='float32')
    
    return result

# Call the function to ensure it works correctly
stu_ans = engagement_model()
print(stu_ans.head())
print(f'Length of output: {len(stu_ans)}')
print()


# Assertions to check the output
assert isinstance(stu_ans, pd.Series), "Your function should return a pd.Series."
assert len(stu_ans) == 2309, "Your series is of incorrect length: expected 2309"
assert np.issubdtype(stu_ans.index.dtype, np.integer), "Your answer pd.Series should have an index of integer type representing video id."

id
9240    0.01
9241    0.06
9242    0.02
9243    0.87
9244    0.06
dtype: float32
Length of output: 2309



In [32]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

def engagement_model():
    # Load the data
    train_df = pd.read_csv('assets/train.csv')
    test_df = pd.read_csv('assets/test.csv')

    # Separate the features and target variable
    X = train_df.drop(columns=['engagement', 'id'])  # Drop 'engagement' and 'id' columns
    y = train_df['engagement'].astype(float)  # Ensure target is float for regression

    # Split into training and validation sets
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0)

    # Preprocess the data: Scale the features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)
    test_scaled = scaler.transform(test_df.drop(columns=['id']))  # Ensure 'id' is dropped

    # Train a Random Forest Regressor model with GridSearchCV
    param_grid = {
        'n_estimators': [50, 100],
        'max_depth': [10, None],
        'min_samples_split': [2, 5],
        'min_samples_leaf': [1, 2],
        'bootstrap': [True]
    }
    grid_search = GridSearchCV(RandomForestRegressor(random_state=0), param_grid, scoring='neg_mean_squared_error', cv=5)
    grid_search.fit(X_train_scaled, y_train)

    # Best model from GridSearchCV
    best_model = grid_search.best_estimator_

    # Predict on the validation set to check performance
    y_val_pred = best_model.predict(X_val_scaled)
    print('Validation MSE:', mean_squared_error(y_val, y_val_pred))

    # Predict on the test set
    test_pred = best_model.predict(test_scaled)


    # Ensure the length of predictions matches the number of test samples
    len(test_pred) == len(test_df)

    # Create the result series with video IDs and set the data type to float32
    result = pd.Series(test_pred, index=test_df['id'], dtype='float32')
    
    return result

# Call the function to ensure it works correctly
stu_ans = engagement_model()
print(stu_ans.head())
print(f'Length of output: {len(stu_ans)}')

# Assertions to check the output
assert isinstance(stu_ans, pd.Series), "Your function should return a pd.Series."
assert len(stu_ans) == 2309, "Your series is of incorrect length: expected 2309"
assert np.issubdtype(stu_ans.index.dtype, np.integer), "Your answer pd.Series should have an index of integer type representing video id."

Validation MSE: 0.05651727490465595
id
9240    0.010000
9241    0.049167
9242    0.020167
9243    0.889833
9244    0.051333
dtype: float32
Length of output: 2309
